<h3>03. 소프트맥스 회귀의 비용함수 구현하기</h3>

In [13]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)


<h4>1. 파이토치로 소프트맥스의 비용함수 구현하기</h4>


In [14]:
z =torch.FloatTensor([1,2,3])

In [15]:
hypothesis = F.softmax(z,dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [16]:
#위 원소의 값이 1인지 확인
hypothesis = F.softmax(z,dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [17]:
hypothesis.sum()

tensor(1.)

In [18]:
z = torch.rand(3, 5, requires_grad=True)


In [19]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


이제 각 행의 원소들의 합은 1이 되는 텐서로 변환되었습니다. 소프트맥스 함수의 출력값은 결국 예측값입니다. 즉, 위 텐서는 3개의 샘플에 대해서 5개의 클래스 중 어떤 클래스가 정답인지를 예측한 결과입니다.

이제 각 샘플에 대해서 임의의 레이블을 만듭니다

In [20]:
y = torch.randint(5, (3,)).long() #5:5가지의수 ,(3,):차원의 수
print(y)

tensor([0, 2, 1])


In [21]:
#모든 원소가 0의 값을 가진 3x 5텐서 생성
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1,y.unsqueeze(1),1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

위의 연산에서 어떻게 원-핫 인코딩이 수행되었는지 보겠습니다. 우선, torch.zeros_like(hypothesis)를 통해 모든 원소가 0의 값을 가진 3 × 5 텐서를 만듭니다. 그리고 이 텐서는 y_one_hot에 저장이 된 상태입니다.

두번째 줄을 해석해봅시다. y.unsqueeze(1)를 하면 (3,)의 크기를 가졌던 y 텐서는 (3 × 1) 텐서가 됩니다. 즉, 다시 말해서 y.unsqueeze(1)의 결과는 아래와 같습니다.

In [22]:
print(y.unsqueeze(1))

tensor([[0],
        [2],
        [1]])


그리고 scatter의 첫번째 인자로 dim=1에 대해서 수행하라고 알려주고, 세번째 인자에 숫자 1을 넣어주므로서 두번째 인자인 y_unsqeeze(1)이 알려주는 위치에 숫자 1을 넣도록 합니다. 앞서 텐서 조작하기 2챕터에서 연산 뒤에 _를 붙이면 In-place Operation (덮어쓰기 연산)임을 배운 바 있습니다. 이에 따라서 y_one_hot의 최종 결과는 결국 아래와 같습니다.

In [23]:
cost = (y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


<h4>2.파이토치로 소프트맥스의 비용함수 구현하기</h4>


<h5>1. F.softmax() + torch.log() = F.log_softmax()</h5>

In [25]:
#앞서 소프트맥스함수의 결과에 로그를 씌울 대 소프트맥수의 출력을 로그합수의 입력으로 사용했다.
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [26]:
#파이토치는 두개의 함수를 결합한 F.log_softmax()를 제공한다.
F.log_softmax(z,dim=1)


tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

<h4>2. F.log_softmax() + F.nll_loss() = F.cross_entropy()</h4>

In [27]:
# 앞서 로우-레벨로 구현한 비용 함수는 다음과 같았습니다.

# Low level
# 첫번째 수식
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [28]:
#위의 수식에서 torch.log(F.softmax(z, dim=1))를 방금 배운 F.log_softmax()로 대체할 수 있습니다.
(y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()


tensor(1.4689, grad_fn=<MeanBackward0>)

In [29]:
#이를 더 간단하게 하면 다음과 같습니다. F.nll_loss()를 사용할 때는 원-핫 벡터를 넣을 필요없이 바로 실제값을 인자로 사용합니다.

F.nll_loss(F.log_softmax(z, dim=1), y)


tensor(1.4689, grad_fn=<NllLossBackward0>)

여기서 nll이란 Negative Log Likelihood의 약자입니다. 위에서 nll_loss는 F.log_softmax()를 수행한 후에 남은 수식들을 수행합니다. 이를 더 간단하게 하면 다음과 같이 사용할 수 있습니다. F.cross_entropy()는 F.log_softmax()와 F.nll_loss()를 포함하고 있습니다.

In [30]:
F.cross_entropy(z, y)


tensor(1.4689, grad_fn=<NllLossBackward0>)